In [3]:
import os
import numpy as np
import tifffile
from tqdm import tqdm


def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.

    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted(
        [
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if (f.endswith(".tiff") or f.endswith(".tif"))
        ]
    )

    tiff_sequence = tifffile.TiffSequence(tiff_files)

    # Get the total number of TIFF files
    total_files = len(tiff_files)

    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)

            # Update progress
            pbar.update(1)

    return np.array(volume)

c:\Users\alberto.vicente\Documents\lunas\.conda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\alberto.vicente\Documents\lunas\.conda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\alberto.vicente\Documents\lunas\.conda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.

    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)

    print("Saving complete.")

In [5]:
from pathlib import Path

folder1 = Path("C:/Users/alberto.vicente/Documents/temporal/volumenes/2/bot_eq")

bot = read_sequence(folder1)

Progress: 100%|██████████| 4425/4425 [00:07<00:00, 576.94it/s]


In [10]:
from pystackreg import StackReg


# Create a functiont that given two slices, registers them and returns the registered slice
def register_slices(args):
    """
    Register two slices using the OpenCV phase correlation method.

    Args:
    slice1 (numpy.ndarray): A 2D array corresponding to a slice.
    slice2 (numpy.ndarray): A 2D array corresponding to a slice.

    Returns:
    numpy.ndarray: A 2D array corresponding to the registered slice.
    """
    print("0")
    slice1, slice2 = args

    return slice2
    t = slice1.dtype
    sr = StackReg(StackReg.TRANSLATION)
    out = sr.register_transform(slice1, slice2).astype(t)

    return out


def stackreg(volume):
    # get middle slice index
    middle = volume.shape[0] // 2

    # Create a list of indexes in pairs
    indexes = [(middle, i + 1) for i in range(volume.shape[0] - 1)]

    volume_reg = np.zeros_like(volume)

    # firslice is the same in both volumes
    volume_reg[0] = volume[0]

    for idx in tqdm(indexes):
        # Register the slices
        slice2_reg = register_slices((volume[idx[0]], volume[idx[1]]))
        # Add the registered slice to the registered volume
        volume_reg[idx[1]] = slice2_reg

    return volume_reg

In [21]:
from concurrent.futures import ProcessPoolExecutor, as_completed


def stackreg_concurrent(volume, n_chunks=5):
    # get middle slice index
    middle = volume.shape[0] // 2

    # Create a list of indexes in pairs
    indexes = [(middle, i + 1) for i in range(volume.shape[0] - 1)]

    volume_reg = np.zeros_like(volume)

    # firslice is the same in both volumes
    volume_reg[0] = volume[0]

    # Split the indexes in chunks
    chunks = np.array_split(indexes, n_chunks)

    # Create a thread pool
    with ProcessPoolExecutor(max_workers=1) as executor:
        counter = 1
        for chunk in chunks:
            print("Registering chunk: ", counter, " of ", n_chunks)
            args = [
                (volume[idx[0]], volume[idx[1]])
                for idx in tqdm(chunk, desc="Creating arguments")
            ]
            # Add the registered slice to the registered volume
            futures = [executor.submit(register_slices, arg) for arg in args]
            result_slices = []
            # Use tqdm for progress bar
            for future in tqdm(as_completed(futures), total=len(futures)):
                result_slices.append(future.result())

    return volume_reg

In [22]:
# compare both methods to check which one is faster
import time

start = time.time()
bot_reg = stackreg(bot[:10])
end = time.time()
first = end - start

start = time.time()
bot_reg = stackreg_concurrent(bot[:10], 1)
end = time.time()
second = end - start

100%|██████████| 9/9 [00:00<00:00, 4500.33it/s]


0
0
0
0
0
0
0
0
0
Registering chunk:  1  of  1


  0%|          | 0/9 [00:00<?, ?it/s]


In [ ]:
# print both times
print(f"First method: {first} seconds")
print(f"Second method: {second} seconds")

First method: 84.90281629562378 seconds
Second method: 95.37798285484314 seconds


In [10]:
# save the registered stack in a folder
folder3 = Path("C:/Users/alberto.vicente/Documents/temporal/volumenes/2")
write_sequence(folder3, "bot_eq_reg", bot_reg)

Saving: 100%|██████████| 4425/4425 [00:13<00:00, 335.89it/s]

Saving complete.


In [20]:
# Save the three images in a folder using tifffile
folder2 = Path("C:/Users/alberto.vicente/Documents/temporal/volumenes/2")
import tifffile as tiff

tiff.imwrite(folder2 / "img1.tif", img1)
tiff.imwrite(folder2 / "img2.tif", img2)
tiff.imwrite(folder2 / "out.tif", out)